In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling, timeframes
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter



In [2]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./half_month_collections.pkl")

In [3]:
print(f"Amount of assets : {len(L)}")
print(f"Amount of meters : {len(M)}")
print(f"Amount of hours : {len(H)} and amount of days : {len(H)/24}")

Amount of assets : 2189
Amount of meters : 62
Amount of hours : 408 and amount of days : 17.0


In [4]:
markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())

In [5]:
type(H[3])

pandas._libs.tslibs.timestamps.Timestamp

In [6]:
def get_possible_dates(date : pd.Timestamp):
    
    if date.hour == 17: # FCR D-2
        return (pd.date_range(date + timedelta(days=1) + timedelta(hours=7), date + timedelta(days = 2) + timedelta(hours = 6), freq='H', tz = "Europe/Oslo"), "D_2")
    elif date.hour == 7: # aFRR
        return (pd.date_range(date + timedelta(hours = 17), date + timedelta(days = 1) + timedelta(hours = 16), freq='H', tz = "Europe/Oslo"), "aFRR")
    elif date.hour == 18: # FCR D-1
        return (pd.date_range(date + timedelta(hours=6), date + timedelta(days = 1) + timedelta(hours = 5), freq='H', tz = "Europe/Oslo"), "D_1")
    else:
        return ([], "No bids")
    
    """elif date.day_of_week == 4: # friday
        if date.hour == 12: #RKOM uke
            # get hours from monday next week until friday next week
            week_dates = pd.date_range(date + timedelta(days=2) + timedelta(hours = 12), periods = 5, freq = 'D', tz = "Europe/Oslo") #RKOM uke weekdays night
            day_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(5, 24)]
            night_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(0, 5)]
            return ((day_hours, night_hours), ("RKOM_uke weekday day", "RKOM_uke weekday night"))
        else:
            return ([], "No bids")
    elif date.day_of_week == 3: #thursday
        if date.hour == 12:
            week_dates = pd.date_range(date + timedelta(days=1), periods = 2, freq='D', tz = "Europe/Oslo") #RKOM uke weekend night
            day_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(5, 24)]
            night_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(0, 5)]
            return ((day_hours, night_hours), ("RKOM_uke weekend day", "RKOM_uke weekend night"))
        else:
            return ([], "No bids")"""

In [44]:
get_possible_dates(pd.Timestamp(year=2023, month=6, day=22, hour=18, minute=0, tz = "Europe/Oslo"))

(DatetimeIndex(['2023-06-23 00:00:00+02:00', '2023-06-23 01:00:00+02:00',
                '2023-06-23 02:00:00+02:00', '2023-06-23 03:00:00+02:00',
                '2023-06-23 04:00:00+02:00', '2023-06-23 05:00:00+02:00',
                '2023-06-23 06:00:00+02:00', '2023-06-23 07:00:00+02:00',
                '2023-06-23 08:00:00+02:00', '2023-06-23 09:00:00+02:00',
                '2023-06-23 10:00:00+02:00', '2023-06-23 11:00:00+02:00',
                '2023-06-23 12:00:00+02:00', '2023-06-23 13:00:00+02:00',
                '2023-06-23 14:00:00+02:00', '2023-06-23 15:00:00+02:00',
                '2023-06-23 16:00:00+02:00', '2023-06-23 17:00:00+02:00',
                '2023-06-23 18:00:00+02:00', '2023-06-23 19:00:00+02:00',
                '2023-06-23 20:00:00+02:00', '2023-06-23 21:00:00+02:00',
                '2023-06-23 22:00:00+02:00', '2023-06-23 23:00:00+02:00'],
               dtype='datetime64[ns, Europe/Oslo]', freq='H'),
 'D_1')

#### The bids doesnt have to be for all hours within a day - can choose to bid for only chosen hours. There is no constraints due to anything but the market constraints. Can be for any number of hours within a day and dont have to be connected hours.

#### The strategy should probably be to bid for the hours where the expected value is highest. If a bid is accepted for an hour in a market it will not be able to bid for the same hour in another market.

In [8]:
def get_possible_bids(possible_hours, available_assets, possible_markets):
    """ Function to find the possible bids for the given input hours, assets and markets. In this version the possible bids are constrained to be for every one of the 24 hours.

    Args:
        possible_hours (DateTimeIndex): list of the possible hours where bids can be placed
        available_assets (list(new_meters.PowerMeter)): list of assets that are not bid in to other markets in the possible hours
        possible_markets (list(final_markets.ReserveMarket)): _description_

    Returns:
        dict: dictionary with the possible bids for each market where the keys are tuples of the market name and the date and the values are tuples of the possible revenues and the assets that can bid in the market
    """
    possible_bids = {}
    date = possible_hours[0].day
    for market in possible_markets:
        if market.direction == "up":
            possible_assets = [asset for asset in available_assets if asset.area == market.area and asset.direction != "down"]
            possible_volumes = np.array([np.sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets]) for hour in possible_hours]) 
            if (possible_volumes > market.min_volume).all(): # this
                possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
                possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
                possible_bids[(market.name, date)] = (possible_revenues, possible_assets)
        elif market.direction == "down":
            possible_assets = [asset for asset in available_assets if asset.area == market.area and asset.direction != "up"]
            possible_volumes = np.array([np.sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets]) for hour in possible_hours]) 
            if (possible_volumes > market.min_volume).all():
                possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
                possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
                possible_bids[(market.name, date)] = (possible_revenues, possible_assets)
        else:
            possible_assets = [asset for asset in available_assets if asset.area == market.area]
            indices = [H.get_loc(ts) for ts in possible_hours]
            directions = np.array(dominant_directions[indices[0]:indices[-1]+1])
            mask_up = directions == "up"
            mask_down = directions == "down"
            possible_up_volumes = np.array([np.sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets]) for hour in possible_hours]) 
            possible_down_volumes = np.array([np.sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets]) for hour in possible_hours]) 
            #print(f"possible_up_volumes: {possible_up_volumes}")
            #print(f"possible_down_volumes: {possible_down_volumes}")
            possible_volumes = np.where(mask_up, possible_up_volumes, possible_down_volumes)
            if (possible_volumes > market.min_volume).all():
                possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
                possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
                possible_bids[(market.name, date)] = (possible_revenues, possible_assets)
    return possible_bids
    


In [9]:
def get_expected_prices_for_hours(possible_hours : pd.Timestamp, area : str, direction : str):
    """ Function to find the expected price for a given hour, area and direction. This function should be updated to be smarter in the future.

    Args:
        hour (pd.Timestamp): hour for which the expected price is calculated
        area (str): area for which the expected price is calculated
        direction (str): direction for which the expected price is calculated

    Returns:
        float: expected price for the given hour, area and direction
    """
    markets_to_check = [market for market in markets_name_dict.values() if market.area == area and market.direction == direction]
    return np.array([np.mean([market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in markets_to_check]) for hour in possible_hours])
    

In [10]:
get_expected_prices_for_hours(H[0:24], "NO1", "up")

array([ 5.08333333,  1.66666667,  1.43333333,  1.43333333,  1.43333333,
        6.38333333, 11.5       , 12.1       , 12.5       , 11.        ,
       12.63333333, 12.62666667, 13.55      , 12.96666667, 12.56666667,
       12.76666667, 11.5       , 11.31666667, 11.23333333, 11.21666667,
       11.3       , 12.60333333, 12.395     , 11.73333333])

In [79]:
def place_bids(possible_hours : [pd.Timestamp], available_assets : dict, market : final_markets.ReserveMarket):
    """ Function to place bids for a given market and set of hours. The bids are placed for every hour in the set of hours. The bids are placed for the assets that are not already bid in to other markets in the given hours. The bids are placed for the assets that are compatible with the given market.

    Args:
        possible_hours (pd.Timestamp]): The time stamps for which the bids are placed
        available_assets (dict): Dictionary with the available assets for each hour
        market (final_markets.ReserveMarket): the market to be bid in to

    Returns:
        tuple (np.array, np.array): The possible revenue of the placed bids for each hour and the assets which are bid for each hour
    """
    expected_prices = get_expected_prices_for_hours(possible_hours, market.area, market.direction)
    hourly_assets = [available_assets[hour] for hour in possible_hours]
    #print(f"expected_pries: {expected_pries}")
    #print(f"hourly_assets: {hourly_assets}")
    if market.direction == "up":
        # check the compatibility for the assets
        possible_assets = [[asset for asset in hourly_assets[h] if asset.area == market.area and asset.direction != "down"] for h in range(len(possible_hours))]
        #print(f"possible_assets: {possible_assets}")
        #get the volumes in the correct direction
        possible_volumes = [sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        possible_volumes = np.where(possible_volumes > [market.min_volume for _ in range(len(possible_volumes))], possible_volumes, 0)
        # get the prices for the given market within the given hours
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        # Calculate the possible revenues
        #print(f"possible_volumes: {possible_volumes}")
        #print(f"possible_prices: {possible_prices}")
        bids_to_be_made = np.where(possible_prices >= expected_prices, possible_prices, 0)
        possible_revenues = np.sum(possible_volumes * bids_to_be_made, axis = 1)
        # Place the bids in the dictionary
        # Must add a better way to store the used assets to the bid
        possible_assets = [assets if possible_revenues[index] > 0 else [] for index, assets in enumerate(possible_assets) ]     

        return (possible_revenues,possible_assets)
    elif market.direction == "down":
        possible_assets = [[asset for asset in hourly_assets[h] if asset.area == market.area and asset.direction != "up"] for h in range(len(possible_hours))]
        possible_volumes = [sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        # all possible volumes has to be higher than the minimum volume for the market
        possible_volumes = np.where(possible_volumes > [market.min_volume for _ in range(len(possible_volumes))], possible_volumes, 0)
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        bids_to_be_made = np.where(possible_prices >= expected_prices, possible_prices, 0)
        possible_revenues = np.sum(possible_volumes * bids_to_be_made, axis = 1)  
        possible_assets = [assets if possible_revenues[index] > 0 else [] for index, assets in enumerate(possible_assets)]     
        return (possible_revenues, possible_assets)
    else:
        possible_assets = [[asset for asset in hourly_assets[h] if asset.area == market.area] for h in range(len(possible_hours))]
        indices = [H.get_loc(ts) for ts in possible_hours]
        directions = np.array(dominant_directions[indices[0]:indices[-1]+1])
        mask_up = directions == "up"
        mask_down = directions == "down"
        possible_up_volumes = [sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        possible_down_volumes = [sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        #print(f"possible_up_volumes: {possible_up_volumes}")
        #print(f"possible_down_volumes: {possible_down_volumes}")
        possible_volumes = np.where(mask_up, possible_up_volumes, possible_down_volumes)
        possible_volumes = np.where(possible_volumes > [market.min_volume for _ in range(len(possible_volumes))], possible_volumes, 0)
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        bids_to_be_made = np.where(possible_prices >= expected_prices, possible_prices, 0)
        possible_revenues = np.sum(possible_volumes * bids_to_be_made, axis = 1)    
        possible_assets = [assets if possible_revenues[index] > 0 else [] for index, assets in enumerate(possible_assets)]     
    
        return (possible_revenues, possible_assets)

In [80]:
def get_possible_bids_unconstrained( H : [pd.Timestamp], L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket]):
    """ Function to find the possible bids for the given input hours, assets and markets. where the possible bids are not constrained to be for every one of the 24 hours.

    Args:
        H (DateTimeIndex): list of the possible hours where bids can be placed
        L ([new_meters.PowerMeter]): list of all assets
        M ([final_markets.ReserveMarket]): list of all markets

    Returns:
        dict: dictionary with the possible bids for each market where the keys are tuples of the market name and the date and the values are tuples of the possible revenues and the assets that can bid in the market
    """
    bid_timeframe = H.append(pd.date_range(H[-1] + timedelta(hours=1), periods=48, freq='H', tz="Europe/Oslo"))
    available_assets = {hour: L.copy() for hour in bid_timeframe}
    bids = {}
    revenue = 0

    for hour in H:
        (possible_hours, market_name) = get_possible_dates(hour)
        if len(possible_hours) != 24:
            #print(f"No bids for {hour}")
            #print(f"possible_hours: {len(possible_hours)}")
            continue

        possible_markets = [m for m in M if market_name in m.name]
        for market in possible_markets:
            (possible_revenues, possible_assets) = place_bids(market=market, available_assets=available_assets, possible_hours=possible_hours)
            revenue += np.sum(possible_revenues)

            for h, bid_hour in enumerate(possible_hours):
                # Select assets with positive revenue for bidding
                assets_to_bid = [asset for asset in possible_assets[h] if possible_revenues[h] > 0]
                # Flatten the list if it's a list of lists
                #assets_to_bid = [item for sublist in assets_to_bid for item in sublist]
                # Store the bid information
                bids[(market.name, bid_hour)] = [asset.meter_id for asset in assets_to_bid]
                # Update available assets
                available_assets[bid_hour] = [asset for asset in available_assets[bid_hour] if asset not in assets_to_bid]

    return bids, revenue, available_assets
            

In [81]:
bids, revenue, av_assets  = get_possible_bids_unconstrained(H[:24], L, M)

In [88]:
revenue

133130.40874478003

In [82]:
markets= [market[0] for market in bids.keys()]
hours = [market[1] for market in bids.keys()]



In [83]:
len(set(markets))

30

In [84]:
len(set(hours))

48

In [85]:
def feasibility_check(bids):
    """ Function to check if the bids are feasible. The bids are feasible if there are no duplicates in the bids for each hour."""
    markets= set([market[0] for market in bids.keys()])
    #print(f"Markets: {markets}")
    hours = set([market[1] for market in bids.keys()])
    #print(f"Hours: {hours}")
    keys = list(bids.keys())
    #print(f"Keys: {keys}")
    hourly_assets = {}
    for hour in hours:
       # print(f"Hour: {hour}")
        hourly_as_list = []
        for market in markets:
            #print(f"Market: {market}")
            if (market, hour) in keys:
                hourly_as_list.append(bids[(market, hour)])
        # Add the flattened list to the dictionary
        hourly_assets[hour] = [item for sublist in hourly_as_list for item in sublist]
    
    #print(hourly_assets)
    for hour in hours:
        if hourly_assets[hour] == []:
            continue
        else:
            """print(f"Hour : {hour}")
            print(f"type of hour object : {type(hour)}")
            print(f"Hourly assets : {hourly_assets[hour]}")
            print(f"length of hourly assets : {len(hourly_assets[hour])}")"""
            # check if there are duplicates
            if len(hourly_assets[hour]) != len(set(hourly_assets[hour])) and hourly_assets[hour] != []:
                print(f"Duplicate assets in hour {hour}")
                # print the duplicates
                print(f"length of hourly assets : {len(hourly_assets[hour])}")
                print(f"length of set of hourly assets : {len(set(hourly_assets[hour]))}")
                return False
    return hourly_assets

In [86]:
h_a = feasibility_check(bids)

In [87]:
[len(h_a[h]) for h in set(hours)]

[2171,
 2171,
 2171,
 2171,
 1661,
 2171,
 0,
 2061,
 2171,
 1661,
 2171,
 1661,
 1661,
 1661,
 1661,
 2171,
 2171,
 2171,
 1661,
 2171,
 2171,
 2171,
 400,
 2171,
 1894,
 1661,
 1661,
 2171,
 2171,
 0,
 1661,
 1661,
 2171,
 2171,
 2171,
 2171,
 1661,
 2171,
 400,
 2171,
 1661,
 1661,
 2171,
 2171,
 1661,
 1661,
 1661,
 1661]

In [ ]:
"""def feasibility_check(possible_bids):
  # Initialize a dictionary to hold counts of integers for each timestamp
    timestamp_counts = defaultdict(lambda: defaultdict(int))

    # Iterate over the original dictionary to populate timestamp_counts
    for (market, hour), assets in possible_bids.items():
        if len(assets) == 0:
            continue
        for asset in assets:
            timestamp_counts[hour][asset] += 1

    # Identify common integers for each timestamp
    for hour, counts in timestamp_counts.items():
        common_assets = {asset.meter_id for asset, count in counts.items() if count > 1}
        if common_assets:
            print(f"Common assets at hour {hour}: {common_assets}")
        else:
            print(f"No common assets at hour {hour}")"""
        
   
        